# Select the segment to be analyzed for LFP and MUA simultaneously.
It must present high power values in these frequency bands ( 0 - 200 Hz for LFP and 300 - 5000 Hz for MUA )

##### 01 - Cutting the segment
##### 02 - Convert to Voltage
##### 03 - Repair electrode saturation
##### 04 - Identify happy friends
##### 05 - Identify damaged electrodes
##### 06 - Parameterize for tissue position identification
##### 07 - Spike detection
##### 08 - CSD
##### 09 - Save the data in jld and hdf5, in raw data and filtered data

In [2]:
# ~~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ constantes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
limupper = 0.6; # GB, for whole file segmentation in uInt8
limite = 15; # maximum percentage of segment that can be saturated
HV = 2500; # upper voltage limit in μV
lV = -2500; # lower voltage limit in μV
method = 1; # method to repair saturation ( 1, 2 or 3 )
ΔT = 400; # ms for the temporary variation
window = 5; # ms for event detection
bit = 1; # window sliding milliseconds
distance = 0.3; # ms maximum distance between events to be discarded
thr = -60; # µV static threshold
mid = 0.6; # Value for plotting the metrics for ACD
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

In [ ]:
# ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~~ paquetes ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
using JLD
using Suppressor
using Plots
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

In [3]:
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
@suppress begin
    PATHFunctions = 
    "/home/isabel/Dropbox/git-repos/Hippocampus-HDMEA/RomeroI-notebooks/ZapfeModifications";
    push!( LOAD_PATH, PATHFunctions );
    using MeaTools
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

In [ ]:
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
FILEBRW = "/run/media/isabel/Data/Hipocampo/Espontanea/01-06-2022/BRWs/Phase_10.brw";
Variables, FILEPATHS, FILEVARS = GetVarsHDF5( FILEBRW );
PATHS = load( FILEPATHS )[ "PATHS" ];
PATHMAIN = PATHS[ "PATHMAIN" ];
PATHRAW = joinpath( PATHMAIN, "Segments" ); mkpath( PATHRAW ); 
PATHS = SavePathsFinal( PATHS );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

In [ ]:
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
σ = ChunkSizeSpace( Variables, limupper ); # Number of segments to cut
n0s = length( string( σ ) );
@time for n in 1:σ
    BIN = OneSegment( Variables, n, σ );
    BINFILE = joinpath( PATHRAW, string( "S", lpad( n, n0s, "0" ), ".jld" ) );
    save( BINFILE, "data", BIN );
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

In [4]:
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
FILEPATHS = "/run/media/isabel/Data/Hipocampo/Espontanea/01-06-2022/Phase_10/Info/Paths.jld";
PATHS = load( FILEPATHS )[ "PATHS" ];
FILEVARS = PATHS[ "FILEVARS" ];
PATHRAW = PATHS[ "PATHRAW" ];
PATHMAIN = PATHS[ "PATHMAIN" ];
PATHINFO = PATHS[ "PATHINFO" ];
Variables = load( FILEVARS )[ "Variables" ];
FILESRAW = SearchDir( PATHRAW, ".jld" );
PATHFIGURES = joinpath( PATHMAIN, "Figures" ); mkpath( PATHFIGURES );
PATHFIGSGENERAL = joinpath( PATHFIGURES, "General" ); mkpath( PATHFIGSGENERAL );
PATHCSDA = joinpath( PATHMAIN, "CSDA" ); mkpath( PATHCSDA );
PATHVOLT = joinpath( PATHMAIN, "Voltage" ); mkpath( PATHVOLT );
PATHFILTERS = joinpath( PATHMAIN, "Filters" ); mkpath( PATHFILTERS );
parameters = Dict(
            "window"    => Ms2Frs( window, Variables ),
            "bit"       => Ms2Frs( bit, Variables ),
            "distance"  => Ms2Frs( distance, Variables ),
            "ΔT"        => Ms2Frs( ΔT, Variables );
        );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

In [10]:
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
nFiles = length( FILESRAW );
n = 45
BINRAW = load( FILESRAW[ n ] )[ "data" ];
BINNAME = split( basename( FILESRAW[ n ] ), "." )[ 1 ];
BINRAW = Digital2Analogue( Variables, BINRAW );
BINRAW, _, _, _ = FixSaturation( 
    BINRAW, limite, HV, lV, method );
save( joinpath( PATHVOLT, string( BINNAME, ".jld" ) ), "RAW", Float16.( BINRAW ) );
S = PFT_mt( Variables, BINRAW );
Ws, AE, BINMUA, BINLFP, metrics = AllWs( Variables, BINRAW, parameters, thr );
save( joinpath( 
    PATHFILTERS, string( BINNAME, ".jld") ), "LFP", BINLFP, "MUA", BINMUA );
BINMUA = [ ]; BINLFP = [ ];
W0NTotal, PF = W0Plots( Ws, BINRAW, mid, metrics );
savefig( PF, joinpath( PATHFIGSGENERAL, string( BINNAME, "W0NPlots.png" ) ) );
savefig( PF, joinpath( PATHFIGSGENERAL, string( BINNAME, "W0NPlots.svg" ) ) );
W = Dict(
    "DirectWeights"  => Ws,
    "DetectedEvents" => AE,
    "labels"         => metrics,
    "LogWeights"     => W0NTotal
);
save( joinpath( 
    PATHINFO, string( BINNAME, "_Analitics.jld" ) ), "Spectrum", S, "Weights", W );
S = [ ]; Ws = [ ]; AE = [ ]; metrics = [ ]; W0NTotal = [ ]; W = [ ];
GST, GS, CSD = CSDAZapfe( BINRAW );
save( joinpath( PATHCSDA, string( 
    BINNAME, "_Partial.jld" ) ), "GaussSmoothTemp", GST, "GaussSmoothFin", GS );
GST = [ ]; GS = [ ];
save( joinpath( PATHCSDA, string( BINNAME, "_CSD.jld" ) ), "CSD", CSD ); CSD = [ ];
println( BINNAME, " Done" );
GC.gc( );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

22 channels repaired with the method: 
Random values taken from the same channel
12 discarted channels.
S45 Done
